py4cytoscapeを利用してトランザクションデータを座標変換
fromとtoをのつながりを持つグラフをcytoscapeに書かせて、登場したアカウントすべてに座標を与えて座標のtxtファイルを獲得

また、valueの数値でフィルタリング

In [1]:

#バージョン確認
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.5.0'}

In [2]:
##sample.csvの読み取り
import numpy as nm
import pandas as pd
import os

#sample.csvのあるディレクトリを指定
os.chdir("/Users/sinco/Desktop/python")

#csvファイルの読み取り
df =pd.read_csv('z.csv')

#読み込む行数の指定 df2は特定行を抽出した表
df = df[1:1000]

取引量(value)が0の列を削除する

In [3]:
#csvファイルの'value'が0より大きい列だけを抽出する
df_mask=df['value']>0
df = df[df_mask]

In [4]:
#指定したアカウントが含まれる行を抽出したdataframeの作成
account = ""
sourcefilter = df["source"].str.contains(account)
targetfilter = df["target"].str.contains(account)
df = pd.concat([df[targetfilter],df[sourcefilter]],axis=0)
print(df)

                                         source  \
1    0x2b9a95c4f3924465fdbcded0a79cbd879830a04f   
3    0x49b667e8edccda3116bb00c189803070ad48466f   
7    0x28c6c06298d514db089934071355e5743bf21d60   
8    0x876eabf441b2ee5b5b0554fd502a8e0600950cfa   
9    0xbcda8d4178a5d63b38cb144715312a74e845ed95   
..                                          ...   
991  0xe021c7b6512247cd5438f25c116455ce761f44ad   
992  0xb345c2f075411cb5f9cd630169b9de6511e24484   
994  0xd1f73b32bc53c02e6cd00e1fbba8ac46330fd3c5   
995  0xfcb001eb6c1873db490e341b806d635a8a9e3d30   
996  0xa81e866e8e92ca16fce4d49d208e59289bf96a68   

                                         target      value       timestamp  \
1    0x28c6c06298d514db089934071355e5743bf21d60   1.630654  20210724180256   
3    0x28c6c06298d514db089934071355e5743bf21d60   0.999433  20210724180256   
7    0xf3d29603b0ae955d4126a5500fb10497758cb8ff   2.068440  20210724180256   
8    0x6f62540d66d45c2d76369c565dc243ce16b8a0f9   7.497594  20210724180256 

In [5]:
#sample.csvからfrom列とto列を抽出(listではない)
fr =df["source"]
to =df["target"]

#list化
frlist = fr.values.tolist()
tolist = to.values.tolist()
print(len(frlist))
print(len(tolist))

frlist1 = fr.values.tolist()

#from,toにあるすべてのアカウント名のリスト(順不同)
#これやるとfrlist1が変形されてしまう
al = []
al = frlist1
al.extend(tolist)
al1=[]
al1=list(set(al))

print(len(al1))

1054
1054
701


sourceとtargetに登場するアカウントは重複が多い

In [6]:
#グラフの表示
import py4cytoscape as p4c
nodes = pd.DataFrame(data={'id':al1})
edges = pd.DataFrame(data={'source':frlist,'target':tolist})
p4c.create_network_from_data_frames(nodes, edges, title="my first network", collection="DataFrame Example")

Applying default style...
Applying preferred layout


124

In [7]:
#指定したアカウントが含まれる行を抽出したdataframeの作成
account = '0xea67'
sourcefilter = df["source"].str.contains(account)
targetfilter = df["target"].str.contains(account)
df_filtered = pd.concat([df[targetfilter],df[sourcefilter]],axis=0)
print(df_filtered)


                                         source  \
178  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
180  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
181  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
182  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
183  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
..                                          ...   
435  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
436  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
437  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
439  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
443  0xea674fdde714fd979de3edf0f56aa9716b898ec8   

                                         target     value       timestamp  \
178  0x685d05f64df5fc7801e75f28c3d0e93971e1202f  0.500710  20210724180304   
180  0x08ea38f58a25a080d65e3a00057d27d3035342f7  0.100004  20210724180304   
181  0x8a5e6c1e3de17e924aa9467587b5d017e5c8627b  0.100184  20210724180304   
182  0xb49c9e9f8dd738bb61bf3ea87b626175177e6a29  0.100098  20210724180304   
18

In [8]:
#ノードの座標の取得から表の作成
'''
n = p4c.get_node_position() #全ノードの座標取得
n['index'] = n.index
n.columns = ['x','y','A']
n['x'] = n['x']/1000
n['y'] = n['y']/1000

#A列(account名)を脱落
n.drop(columns=["A"],inplace = True)
print(n)

#テキストファイルにして出力
n.to_csv("cytoTDA_0xea67.txt", sep=" ", index=False)
'''

'\nn = p4c.get_node_position() #全ノードの座標取得\nn[\'index\'] = n.index\nn.columns = [\'x\',\'y\',\'A\']\nn[\'x\'] = n[\'x\']/1000\nn[\'y\'] = n[\'y\']/1000\n\n#A列(account名)を脱落\nn.drop(columns=["A"],inplace = True)\nprint(n)\n\n#テキストファイルにして出力\nn.to_csv("cytoTDA_0xea67.txt", sep=" ", index=False)\n'